In [ ]:
import pandas as pd
import numpy as np
import pickle
from sklearn.metrics import classification_report

def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    try:
        with open(name + '.pkl', 'rb') as f:
            return pickle.load(f)    
    except FileNotFoundError as e:
        return False;

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


.

.

**NN ANSAMBLE**

.

.


.


In [ ]:
from sklearn.model_selection import train_test_split 

data = pd.read_csv('/content/drive/MyDrive/SIAP/data/prepared/final_vectors_with_buckets.csv')
data = data.dropna(subset=['REASONDESCRIPTION'])

train, test = train_test_split(data, test_size=0.2, random_state = 42)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, BatchNormalization
from keras.optimizers import Adam
from keras.utils import to_categorical
import tensorflow as tf
from keras.callbacks import ModelCheckpoint 
from datetime import datetime 
from keras.metrics import BinaryAccuracy, Precision, Recall, AUC

num_epochs = 20
num_batch_size = 64
# Notes:
# BatchNormalization recall from 0.5 to 0.8
# tf.keras.losses.CategoricalCrossentropy(from_logits=True) 0.95 recall, loss 0 at start...
METRICS = [
      BinaryAccuracy(name='accuracy'),
      Precision(name='precision'),
      Recall(name='recall'),
      AUC(name='auc'),
      AUC(name='prc', curve='PR'), # precision-recall curve
]


def build_model(input_dimension):
    model = Sequential()
    model.add(Dense(1024, input_shape=(input_dimension,)))
    model.add(BatchNormalization())
    model.add(Activation('selu'))
    model.add(Dropout(0.2))
    model.add(Dense(128))
    model.add(BatchNormalization())
    model.add(Activation('selu'))
    model.add(Dropout(0.2))
    model.add(Dense(64))
    model.add(BatchNormalization())
    model.add(Activation('selu'))
    model.add(Dropout(0.2))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    # Compile the model
    model.compile(loss=tf.keras.losses.BinaryCrossentropy() , metrics=METRICS, optimizer='adam')

    # model.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True) , metrics=METRICS, optimizer='adam') 
    return model

nn_model = build_model(train.shape[1])
nn_model.summary()

Model: "sequential_27"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_108 (Dense)            (None, 1024)              1462272   
_________________________________________________________________
batch_normalization_81 (Batc (None, 1024)              4096      
_________________________________________________________________
activation_108 (Activation)  (None, 1024)              0         
_________________________________________________________________
dropout_81 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense_109 (Dense)            (None, 128)               131200    
_________________________________________________________________
batch_normalization_82 (Batc (None, 128)               512       
_________________________________________________________________
activation_109 (Activation)  (None, 128)             

In [ ]:

death_reasons = np.sort(data['REASONDESCRIPTION'].unique())

# death_reasons = ['Cardiac Arrest', 'Chronic congestive heart failure (disorder)',\
#                  'Chronic obstructive bronchitis (disorder)', 'Malignant neoplasm of breast (disorder)',\
#                  'Malignant tumor of colon', 'Myocardial Infarction', 'Non-small cell lung cancer (disorder)', 'Pulmonary emphysema (disorder)', 'Stroke']

for index, reason in enumerate(death_reasons):
  train['IS_SELECTED_DEATH_REASON'] = train['REASONDESCRIPTION'] == reason

  y = train['IS_SELECTED_DEATH_REASON'].to_numpy() * 1
  X = train[train.columns.difference(['PATIENT', 'IS_SELECTED_DEATH_REASON', 'REASONDESCRIPTION'])].to_numpy()

  print("-"*80)
  print(str(index) + " ------ " + reason + " " + str(len(np.nonzero(y)[0])))

  nn_model = build_model(X.shape[1])
  nn_model.fit(X, y, batch_size=num_batch_size, epochs=num_epochs, validation_split=0.1, verbose=1)
  nn_model.save('/content/drive/MyDrive/SIAP/models/ansamble/' + str(index) + '.h5')
  print("-"*80)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


--------------------------------------------------------------------------------
0 ------ Alzheimer's disease (disorder) 21
Epoch 1/20
216/216 [==============================] - 4s 11ms/step - loss: 0.4670 - accuracy: 0.8103 - precision: 8.7076e-04 - recall: 0.1881 - auc: 0.4892 - prc: 9.8024e-04 - val_loss: 0.0525 - val_accuracy: 0.9993 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.9987 - val_prc: 0.1891
Epoch 2/20
216/216 [==============================] - 2s 7ms/step - loss: 0.0411 - accuracy: 0.9977 - precision: 0.0512 - recall: 0.0319 - auc: 0.9261 - prc: 0.0751 - val_loss: 0.0086 - val_accuracy: 0.9993 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 1.0000 - val_prc: 1.0000
Epoch 3/20
216/216 [==============================] - 2s 7ms/step - loss: 0.0141 - accuracy: 0.9985 - precision: 0.5146 - recall: 0.3033 - auc: 0.9799 - prc: 0.2659 - val_loss: 0.0032 - val_accuracy: 1.0000 - val_precision: 1.0000 - val_recall: 1.0000 - val_auc: 1.0000 - val

In [ ]:
for index, reason in enumerate(death_reasons):
  test['IS_SELECTED_DEATH_REASON'] = test['REASONDESCRIPTION'] == reason

  y = test['IS_SELECTED_DEATH_REASON'].to_numpy() * 1
  X = test[test.columns.difference(['PATIENT', 'IS_SELECTED_DEATH_REASON', 'REASONDESCRIPTION'])].to_numpy()

  print("-"*80)
  print(str(index) + " ------ " + reason + " " + str(len(np.nonzero(y)[0])))

  nn_model = tf.keras.models.load_model('/content/drive/MyDrive/SIAP/models/ansamble/' + str(index) + '.h5', compile=True)
  y_pred = nn_model.predict(X, batch_size=64, verbose=1)
  y_pred = (y_pred > 0.5)
  print(classification_report(y, y_pred, zero_division=0))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


--------------------------------------------------------------------------------
0 ------ Alzheimer's disease (disorder) 6
60/60 [==============================] - 0s 2ms/step
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3829
           1       0.30      0.50      0.37         6

    accuracy                           1.00      3835
   macro avg       0.65      0.75      0.69      3835
weighted avg       1.00      1.00      1.00      3835

--------------------------------------------------------------------------------
1 ------ Burn injury(morphologic abnormality) 33
60/60 [==============================] - 0s 1ms/step
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      3802
           1       0.00      0.00      0.00        33

    accuracy                           0.99      3835
   macro avg       0.50      0.50      0.50      3835
weighted avg       0.98      0.99      0.9

In [ ]:
from sklearn import tree


death_reasons = np.sort(data['REASONDESCRIPTION'].unique())

# death_reasons = ['Cardiac Arrest', 'Chronic congestive heart failure (disorder)',\
#                  'Chronic obstructive bronchitis (disorder)', 'Malignant neoplasm of breast (disorder)',\
#                  'Malignant tumor of colon', 'Myocardial Infarction', 'Non-small cell lung cancer (disorder)', 'Pulmonary emphysema (disorder)', 'Stroke']

for index, reason in enumerate(death_reasons):
  train['IS_SELECTED_DEATH_REASON'] = train['REASONDESCRIPTION'] == reason

  y = train['IS_SELECTED_DEATH_REASON'].to_numpy() * 1
  X = train[train.columns.difference(['PATIENT', 'IS_SELECTED_DEATH_REASON', 'REASONDESCRIPTION'])].to_numpy()

  print("-"*80)
  print(str(index) + " ------ " + reason + " " + str(len(np.nonzero(y)[0])))

  dtree_clf = tree.DecisionTreeClassifier(criterion = 'entropy', min_samples_split = 2, min_samples_leaf = 1, max_depth= 3)
  dtree_clf = dtree_clf.fit(X, y)
  save_obj(dtree_clf, '/content/drive/MyDrive/SIAP/models/ansamble/trees/' + str(index))

  print("-"*80)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


--------------------------------------------------------------------------------
0 ------ Alzheimer's disease (disorder) 21
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
1 ------ Burn injury(morphologic abnormality) 145


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
2 ------ COVID-19 1531


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


...

--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
27 ------ Stroke 308


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
28 ------ Sudden Cardiac Death 3077


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


--------------------------------------------------------------------------------


In [ ]:
for index, reason in enumerate(death_reasons):
  test['IS_SELECTED_DEATH_REASON'] = test['REASONDESCRIPTION'] == reason

  y = test['IS_SELECTED_DEATH_REASON'].to_numpy() * 1
  X = test[test.columns.difference(['PATIENT', 'IS_SELECTED_DEATH_REASON', 'REASONDESCRIPTION'])].to_numpy()

  print("-"*80)
  print(str(index) + " ------ " + reason + " " + str(len(np.nonzero(y)[0])))

  dtree_clf = load_obj('/content/drive/MyDrive/SIAP/models/ansamble/trees/' + str(index))
  print(classification_report(y, dtree_clf.predict(X)))


--------------------------------------------------------------------------------
0 ------ Alzheimer's disease (disorder) 6
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3829
           1       0.00      0.00      0.00         6

    accuracy                           1.00      3835
   macro avg       0.50      0.50      0.50      3835
weighted avg       1.00      1.00      1.00      3835

--------------------------------------------------------------------------------
1 ------ Burn injury(morphologic abnormality) 33
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      3802
           1       0.00      0.00      0.00        33

    accuracy                           0.99      3835
   macro avg       0.50      0.50      0.50      3835
weighted avg       0.98      0.99      0.99      3835

--------------------------------------------------------------------------------
2 ------ COV

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3779
           1       0.79      0.82      0.81        56

    accuracy                           0.99      3835
   macro avg       0.90      0.91      0.90      3835
weighted avg       0.99      0.99      0.99      3835

--------------------------------------------------------------------------------
6 ------ Concussion injury of brain 95
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      3740
           1       0.00      0.00      0.00        95

    accuracy                           0.98      3835
   macro avg       0.49      0.50      0.49      3835
weighted avg       0.95      0.98      0.96      3835

--------------------------------------------------------------------------------
7 ------ Death due to acute respiratory failure 6
              precision    recall  f1-score   support

           0       1.00      1.00      1

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-v

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3797
           1       0.97      0.76      0.85        38

    accuracy                           1.00      3835
   macro avg       0.98      0.88      0.93      3835
weighted avg       1.00      1.00      1.00      3835

--------------------------------------------------------------------------------
13 ------ Malignant tumor of colon 23
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3812
           1       0.88      1.00      0.94        23

    accuracy                           1.00      3835
   macro avg       0.94      1.00      0.97      3835
weighted avg       1.00      1.00      1.00      3835

--------------------------------------------------------------------------------
14 ------ Meningomyelocele (disorder) 8
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      38

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3795
           1       0.95      1.00      0.98        40

    accuracy                           1.00      3835
   macro avg       0.98      1.00      0.99      3835
weighted avg       1.00      1.00      1.00      3835

--------------------------------------------------------------------------------
19 ------ Overlapping malignant neoplasm of colon 14
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3821
           1       0.82      0.64      0.72        14

    accuracy                           1.00      3835
   macro avg       0.91      0.82      0.86      3835
weighted avg       1.00      1.00      1.00      3835

--------------------------------------------------------------------------------
20 ------ Pneumonia 16
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3819

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3820
           1       1.00      1.00      1.00        15

    accuracy                           1.00      3835
   macro avg       1.00      1.00      1.00      3835
weighted avg       1.00      1.00      1.00      3835

--------------------------------------------------------------------------------
25 ------ Sepsis of Pseudomonas 0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3835

    accuracy                           1.00      3835
   macro avg       1.00      1.00      1.00      3835
weighted avg       1.00      1.00      1.00      3835

--------------------------------------------------------------------------------
26 ------ Small cell carcinoma of lung (disorder) 6
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3829
           1       1.00      1.00      1.00

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-v